# Airflow Dag

## Import Libraries

In [4]:
pip install "apache-airflow[celery]==2.2.4"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is not installed.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
sphinx 4.0.1 requires MarkupSafe<2.0, but you have markupsafe 2.0.1 which is incompatible.


  Using cached importlib_resources-5.4.0-py3-none-any.whl (28 kB)
  Created wheel for Flask-JWT-Extended: filename=Flask_JWT_Extended-3.25.1-py2.py3-none-any.whl size=21599 sha256=b7a7519230fd556856a5977e358b9c0d45f0302d2e5f2cccfc5079705cb84853
  Stored in directory: c:\users\indra\appdata\local\pip\cache\wheels\00\60\4a\7fc484ed75cc2baea756f0a3d07aa2065d09688d940c6fac1e
  Created wheel for flask-login: filename=Flask_Login-0.4.1-py2.py3-none-any.whl size=15936 sha256=80f374f000f8d9ee473ae6cdb878029ab7b1b90c0cbaba136a27be8a240276d2
  Stored in directory: c:\users\indra\appdata\local\pip\cache\wheels\ec\8e\42\16110a8b4f17e4ef4bd0abd42969fc1fc38229348de9f071df
  Created wheel for python-nvd3: filename=python_nvd3-0.15.0-py3-none-any.whl size=38156 sha256=6313958bb4500b376ca43ba195fc8d89669079e378ce95886040cc3ce9664cd9
  Stored in directory: c:\users\indra\appdata\local\pip\cache\wheels\8c\92\61\c1d567a155a1f15d2e1a633eef32ed3a7c7906c323d49fa3a2
  Created wheel for python-slugify: filenam

In [6]:
from airflow import DAG
from datetime import datetime
from io import StringIO

from airflow.operators.python_operator import PythonOperator
from airflow.operators import S3KeySensor
from airflow.operators import S3Hook

ModuleNotFoundError: No module named 'termios'

In [ ]:


# [START howto_operator_athena_env_variables]
S3_BUCKET = getenv("S3_BUCKET", "test-bucket")
S3_KEY = getenv("S3_KEY", "key")
ATHENA_TABLE = getenv("ATHENA_TABLE", "test_table")
ATHENA_DATABASE = getenv("ATHENA_DATABASE", "default")
# [END howto_operator_athena_env_variables]

SAMPLE_DATA = """"Magalu",200000
"Polishop",250000
"Mariza",3000000
"""
SAMPLE_FILENAME = 'itau_sample.csv'


@task(task_id='setup__add_sample_data_to_s3')
def add_sample_data_to_s3():
    s3_hook = S3Hook()
    s3_hook.load_string(SAMPLE_DATA, f'{S3_KEY}/{ATHENA_TABLE}/{SAMPLE_FILENAME}', S3_BUCKET, replace=True)


@task(task_id='teardown__remove_sample_data_from_s3')
def remove_sample_data_from_s3():
    s3_hook = S3Hook()
    if s3_hook.check_for_key(f'{S3_KEY}/{ATHENA_TABLE}/{SAMPLE_FILENAME}', S3_BUCKET):
        s3_hook.delete_objects(S3_BUCKET, f'{S3_KEY}/{ATHENA_TABLE}/{SAMPLE_FILENAME}')


@task(task_id='query__read_results_from_s3')
def read_results_from_s3(query_execution_id):
    s3_hook = S3Hook()
    if s3_hook.check_for_key(f'{S3_KEY}/{query_execution_id}.csv', S3_BUCKET):
        file_obj = s3_hook.get_conn().get_object(Bucket=S3_BUCKET, Key=f'{S3_KEY}/{query_execution_id}.csv')
        file_content = file_obj['Body'].read().decode('utf-8')
        print(file_content)
    else:
        print('Could not find QueryExecutionId:', query_execution_id)




In [ ]:
QUERY_CREATE_TABLE = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DATABASE}.{ATHENA_TABLE} ( `name` string, `age` int )
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ( 'serialization.format' = ',', 'field.delim' = ','
) LOCATION 's3://{S3_BUCKET}/{S3_KEY}/{ATHENA_TABLE}'
TBLPROPERTIES ('has_encrypted_data'='false')
"""

QUERY_READ_TABLE = f"""
SELECT * from {ATHENA_DATABASE}.{ATHENA_TABLE}
"""

QUERY_DROP_TABLE = f"""
DROP TABLE IF EXISTS {ATHENA_DATABASE}.{ATHENA_TABLE}
"""

with DAG(
    dag_id='example_athena',
    schedule_interval=None,
    start_date=datetime(2021, 1, 1),
    dagrun_timeout=timedelta(minutes=60),
    tags=['example'],
    catchup=False,
) as dag:
    # [START howto_athena_operator_and_sensor]

    # Using a task-decorated function to create a CSV file in S3
    add_sample_data_to_s3 = add_sample_data_to_s3()

    create_table = AthenaOperator(
        task_id='setup__create_table',
        query=QUERY_CREATE_TABLE,
        database=ATHENA_DATABASE,
        output_location=f's3://{S3_BUCKET}/{S3_KEY}',
        sleep_time=30,
        max_tries=None,
    )

    read_table = AthenaOperator(
        task_id='query__read_table',
        query=QUERY_READ_TABLE,
        database=ATHENA_DATABASE,
        output_location=f's3://{S3_BUCKET}/{S3_KEY}',
        sleep_time=30,
        max_tries=None,
    )

    get_read_state = AthenaSensor(
        task_id='query__get_read_state',
        query_execution_id=read_table.output,
        max_retries=None,
        sleep_time=10,
    )

    # Using a task-decorated function to read the results from S3
    read_results_from_s3 = read_results_from_s3(read_table.output)

    drop_table = AthenaOperator(
        task_id='teardown__drop_table',
        query=QUERY_DROP_TABLE,
        database=ATHENA_DATABASE,
        output_location=f's3://{S3_BUCKET}/{S3_KEY}',
        sleep_time=30,
        max_tries=None,
    )

    # Using a task-decorated function to delete the S3 file we created earlier
    remove_sample_data_from_s3 = remove_sample_data_from_s3()

    (
        add_sample_data_to_s3
        >> create_table
        >> read_table
        >> get_read_state
        >> read_results_from_s3
        >> drop_table
        >> remove_sample_data_from_s3
    )
    # [END howto_athena_operator_and_sensor]

    # Task dependencies created via `XComArgs`:
    #   read_table >> get_read_state
    #   read_table >> read_results_from_s3